<a href="https://colab.research.google.com/github/joekrejci/personal_qdrant_projects/blob/main/joes_qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain qdrant_client openai tiktoken

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings

import qdrant_client
import os

In [ ]:
#Create a Qdrant Client

os.environ['QDRANT_HOST'] = "https://{qdrant-url}.eu-central.aws.staging-cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] = "{qdrant_api_key}"

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [ ]:
#Create Collection

os.environ["QDRANT_COLLECTION_NAME"] = "joes-special-collection"

vectors_config = qdrant_client.http.models.VectorParams(
    size=1536,
    distance=qdrant_client.http.models.Distance.COSINE
    )

client.create_collection(
    collection_name= os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `joes-special-collection` already exists!"},"time":0.010772885}'

In [ ]:
#Create Vector Store

os.environ["OPENAI_API_KEY"] = "{open_api_api_key}"

embeddings = OpenAIEmbeddings ()

vector_store = Qdrant(
    client=client, collection_name= os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings=embeddings,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# Add Documents to Vector Store

from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator="/n",
      chunk_size=3000,
      chunk_overlap=600,
      length_function=len
  )

  chunks = text_splitter.split_text(text)
  return chunks

with open('samir_discord.txt') as f:
  raw_text = f.read()


texts = get_chunks(raw_text)

vector_store.add_texts(texts)

['aab53377fa904ad0ac3932639d712814', '774604cb58d24cc194543e72c79baf1a']

In [ ]:
# Plug Vector Store Into Retrieval Chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
query = "Ask me a question here"

response = qa.run(query)

print (response)

NameError: name 'qa' is not defined